In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import seaborn as sns

In [2]:
%cd /home/stepan/kaggle/cars_from_auction

/home/stepan/kaggle/cars_from_auction


In [3]:
%ls

data/  models/  prepare/  result/  review/  test/  train/


In [4]:
%ls data/

processed/  sample_submission.csv  test.csv  train.csv  zipcodes.csv


In [5]:
data_path = 'data/processed/'
data_file = 'transformed_test.csv'

In [6]:
data = pd.read_csv(data_path + data_file, index_col=0)

In [7]:
data

,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price
60314,1.6,small car,2013,manual,136.0,swift,40000,gasoline,suzuki,0.0,30449,6.194405
12566,NaN,coupé,2004,auto,333.0,6er,150000,gasoline,bmw,0.0,45307,6.507278
17760,NaN,station wagon,2006,auto,170.0,e_klasse,150000,diesel,mercedes_benz,0.0,59494,6.131226
8876,NaN,limousine,1999,manual,101.0,astra,150000,gasoline,opel,NaN,25524,NaN
80392,NaN,limousine,1975,manual,54.0,andere,150000,diesel,mercedes_benz,0.0,70794,7.012115
...,...,...,...,...,...,...,...,...,...,...,...,...
93878,1.4,limousine,1999,manual,86.0,corolla,150000,gasoline,toyota,0.0,44339,4.700480
99783,NaN,station wagon,2002,auto,184.0,3er,150000,diesel,bmw,0.0,47574,5.480639
57399,NaN,small car,2005,manual,52.0,fox,100000,gasoline,volkswagen,0.0,50389,4.094345
97106,NaN,bus,2001,manual,151.0,transporter,150000,diesel,volkswagen,0.0,12209,6.835185


In [8]:
zip_data_path = 'data/processed/'
zip_data_file = 'transformed_zipcodes.csv'

In [9]:
zip_data = pd.read_csv(zip_data_path + zip_data_file, index_col=0)

In [10]:
zip_data

,zipcode,city,latitude,longitude,dist
0,19348,Berge bei Perleberg,53.237460,11.870770,2975.142328
1,85309,Pörnbach,48.616700,11.466700,2495.068728
2,24790,"Osterrönfeld Heidkrug, Gemeinde Osterrönfeld",54.275360,9.737535,3040.634291
3,98646,Hildburghausen,50.439501,10.723922,2659.145779
4,27336,"Frankenfeld, Aller",52.769510,9.430780,2873.560797
...,...,...,...,...,...
17370,48465,Isterberg,52.363460,7.160770,2793.208570
17371,14929,Treuenbrietzen Bardenitz,52.070188,12.937977,2878.695733
17372,14913,Treuenbrietzen Bardenitz,52.070188,12.937977,2878.695733
17373,22946,Dahmker,53.587820,10.444710,2980.746419


# Merge

In [11]:
def merge_data(data, zipcodes):
    zipcodes = zipcodes.drop_duplicates(keep='last', subset=['zipcode'])
    mdata = data.reset_index().merge(zipcodes, on="zipcode", how="left").set_index('index')
    return mdata

In [12]:
new_data = merge_data(data, zip_data)

In [13]:
new_data

,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price,city,latitude,longitude,dist
index,,,,,,,,,,,,,,,,
60314,1.6,small car,2013,manual,136.0,swift,40000,gasoline,suzuki,0.0,30449,6.194405,Hannover,52.384470,9.726930,2838.745864
12566,NaN,coupé,2004,auto,333.0,6er,150000,gasoline,bmw,0.0,45307,6.507278,"Essen, Ruhr",51.462488,7.008645,2697.508782
17760,NaN,station wagon,2006,auto,170.0,e_klasse,150000,diesel,mercedes_benz,0.0,59494,6.131226,"Soest, Westfalen",51.566980,8.110620,2724.935583
8876,NaN,limousine,1999,manual,101.0,astra,150000,gasoline,opel,NaN,25524,NaN,Itzehoe,53.926752,9.512700,2998.586043
80392,NaN,limousine,1975,manual,54.0,andere,150000,diesel,mercedes_benz,0.0,70794,7.012115,Filderstadt,48.666700,9.216670,2453.394695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93878,1.4,limousine,1999,manual,86.0,corolla,150000,gasoline,toyota,0.0,44339,4.700480,Dortmund,51.504225,7.483654,2708.690313
99783,NaN,station wagon,2002,auto,184.0,3er,150000,diesel,bmw,0.0,47574,5.480639,Goch,51.679880,6.156480,2708.712243
57399,NaN,small car,2005,manual,52.0,fox,100000,gasoline,volkswagen,0.0,50389,4.094345,"Wesseling, Rheinland",50.833300,6.983330,2632.791287


In [14]:
new_data.describe()

,engine_capacity,registration_year,power,mileage,damage,zipcode,insurance_price,latitude,longitude,dist
count,20086.000000,50000.000000,50000.000000,50000.000000,41773.000000,50000.000000,42594.000000,49952.000000,49952.000000,49952.000000
mean,1.780643,2003.733600,122.107725,125278.300000,0.092931,51196.608000,5.479055,50.873700,9.436050,2684.357513
std,0.470688,7.299557,60.319161,39509.609835,0.290339,25750.460527,1.042473,1.705111,2.068292,179.650818
min,0.800000,1945.000000,25.000000,5000.000000,0.000000,1067.000000,2.302585,47.410100,5.916670,2320.036060
25%,1.400000,2000.000000,75.000000,100000.000000,0.000000,30926.000000,4.605170,49.466700,7.738370,2546.067942
50%,1.800000,2004.000000,110.000000,150000.000000,0.000000,50170.000000,5.438079,51.000000,9.037355,2669.946073
75%,2.000000,2008.000000,150.000000,150000.000000,0.000000,71720.750000,6.234411,52.199974,10.883300,2817.244473
max,3.300000,2016.000000,495.000000,150000.000000,1.000000,99998.000000,10.858807,54.937195,14.970560,3168.285233


In [15]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 60314 to 478
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   engine_capacity    20086 non-null  float64
 1   type               43834 non-null  object 
 2   registration_year  50000 non-null  int64  
 3   gearbox            48021 non-null  object 
 4   power              50000 non-null  float64
 5   model              47671 non-null  object 
 6   mileage            50000 non-null  int64  
 7   fuel               46408 non-null  object 
 8   brand              50000 non-null  object 
 9   damage             41773 non-null  float64
 10  zipcode            50000 non-null  int64  
 11  insurance_price    42594 non-null  float64
 12  city               49952 non-null  object 
 13  latitude           49952 non-null  float64
 14  longitude          49952 non-null  float64
 15  dist               49952 non-null  float64
dtypes: float64(7), int64

In [16]:
new_data.to_csv(data_path + 'merged_' + data_file)